In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "transformed_features_and_target_top50.parquet")
df

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,temp_t-1,wind_speed_t-3,wind_speed_t-2,wind_speed_t-1,precipitation_t-3,precipitation_t-2,precipitation_t-1,target,start_station_id,hour
0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,3.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,A32002,2024-10-28 22:00:00
1,0.0,0.0,3.0,1.0,0.0,0.0,0.0,3.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,A32002,2024-10-28 23:00:00
2,0.0,3.0,1.0,0.0,0.0,0.0,3.0,6.0,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,A32002,2024-10-29 00:00:00
3,3.0,1.0,0.0,0.0,0.0,3.0,6.0,8.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,A32002,2024-10-29 01:00:00
4,1.0,0.0,0.0,0.0,3.0,6.0,8.0,6.0,8.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,A32002,2024-10-29 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404579,16.0,9.0,9.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,M32085,2025-09-30 19:00:00
404580,9.0,9.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,M32085,2025-09-30 20:00:00
404581,9.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,M32085,2025-09-30 21:00:00
404582,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,M32085,2025-09-30 22:00:00


In [4]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2025, 7, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(294163, 683)
(294163,)
(110421, 683)
(110421,)


In [5]:
past_ride_columns = [c for c in X_train.columns if c.startswith("rides_")]
X_train_only_numeric = X_train[past_ride_columns]
X_test_only_numeric = X_test[past_ride_columns]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

rf_model = RandomForestRegressor(
    n_estimators=50,
    max_depth=8,
    min_samples_leaf=5,
    n_jobs=4,       # adjust to your CPU cores
    random_state=42,
    verbose=1
)

print("Training Random Forest...")
rf_model.fit(X_train_only_numeric, y_train)




Training Random Forest...


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  3.0min


✅ Random Forest MAE: 2.1824


[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:  3.5min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.1s finished


In [8]:
rf_preds = rf_model.predict(X_test_only_numeric)
rf_mae = mean_absolute_error(y_test, rf_preds)

print(f"✅ Random Forest MAE: {rf_mae:.4f}")

✅ Random Forest MAE: 2.1824


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.1s finished
